# Pyber Challenge

### 4.3 Loading and Reading CSV files

In [ ]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import dataframe_image as dfi   # This is for save dataframe as png images

# File to Load (Remember to change these)
city_data_to_load = "Resources/city_data.csv"
ride_data_to_load = "Resources/ride_data.csv"

# Read the City and Ride Data
city_data_df = pd.read_csv(city_data_to_load)
ride_data_df = pd.read_csv(ride_data_to_load)

### Merge the DataFrames

In [ ]:
# Combine the data into a single dataset
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Display the data table for preview
pyber_data_df.head()

## Deliverable 1: Get a Summary DataFrame 

In [ ]:
#  1. Get the total rides for each city type
rides_by_city_type = pyber_data_df.groupby("type").count()["ride_id"]
rides_by_city_type

In [ ]:
# 2. Get the total drivers for each city type
drivers_by_city_type = city_data_df.groupby("type").sum()["driver_count"]
drivers_by_city_type

In [ ]:
#  3. Get the total amount of fares for each city type
fares_by_city_type = pyber_data_df.groupby("type").sum()["fare"]
fares_by_city_type

In [ ]:
#  4. Get the average fare per ride for each city type. 
avgfarebyride_by_city_type = fares_by_city_type/rides_by_city_type
avgfarebyride_by_city_type

In [ ]:
# 5. Get the average fare per driver for each city type. 
avgfarebydriver_by_city_type = fares_by_city_type/drivers_by_city_type
avgfarebydriver_by_city_type

In [ ]:
#  6. Create a PyBer summary DataFrame. 
pyber_summary_df = pd.DataFrame(
            {'Total Rides':rides_by_city_type,
             'Total Drivers':drivers_by_city_type,
             'Total Fares':fares_by_city_type,
             'Average Fare per Ride':avgfarebyride_by_city_type,
             'Average Fare per Driver':avgfarebydriver_by_city_type}
            )
pyber_summary_df

In [ ]:
#  7. Cleaning up the DataFrame. Delete the index name
pyber_summary_df.index.name = None

In [ ]:
#  8. Format the columns.
pyber_summary_df['Total Rides'] = pyber_summary_df['Total Rides'].map('{:,}'.format)
pyber_summary_df['Total Drivers'] = pyber_summary_df['Total Drivers'].map('{:,}'.format)
pyber_summary_df['Total Fares'] = pyber_summary_df['Total Fares'].map('${:,.2f}'.format)
pyber_summary_df['Average Fare per Ride'] = pyber_summary_df['Average Fare per Ride'].map('${:,.2f}'.format)
pyber_summary_df['Average Fare per Driver'] = pyber_summary_df['Average Fare per Driver'].map('${:,.2f}'.format)
pyber_summary_df

In [ ]:
# saving the df as a figure
dfi.export(pyber_summary_df, 'analysis/df_pyberSummary.png')

## Deliverable 2.  Create a multiple line plot that shows the total weekly of the fares for each type of city.

In [ ]:
# 1. Read the merged DataFrame
pyber_data_df

In [ ]:
# 2. Using groupby() to create a new DataFrame showing the sum of the fares 
#  for each date where the indices are the city type and date.
fare_by_date_df = pyber_data_df.groupby(["type","date"]).sum()["fare"]
fare_by_date_df

In [ ]:
# 3. Reset the index on the DataFrame you created in #1. This is needed to use the 'pivot()' function.
fare_by_date_df = fare_by_date_df.reset_index()


In [ ]:
# 4. Create a pivot table with the 'date' as the index, the columns ='type', and values='fare' 
# to get the total fares for each type of city by the date. 

fare_by_date_pivottable = pd.pivot_table(fare_by_date_df, values='fare', index='date', columns='type')
fare_by_date_pivottable

In [ ]:
# 5. Create a new DataFrame from the pivot table DataFrame using loc on the given dates, '2019-01-01':'2019-04-29'.

fare_by_date_pivot_df = fare_by_date_pivottable.loc['2019-01-01':'2019-04-29']

In [ ]:
# 6. Set the "date" index to datetime datatype. This is necessary to use the resample() method in Step 8.
fare_by_date_pivot_df.index = pd.to_datetime(fare_by_date_pivot_df.index)

In [ ]:
# 7. Check that the datatype for the index is datetime using df.info()
fare_by_date_pivot_df.info()

In [ ]:
# 8. Create a new DataFrame using the "resample()" function by week 'W' and get the sum of the fares for each week.
resample_fare_by_date_df = fare_by_date_pivot_df.resample('W').sum()
resample_fare_by_date_df

In [ ]:
# 8. Using the object-oriented interface method, plot the resample DataFrame using the df.plot() function. 

# Import the style from Matplotlib.
from matplotlib import style

resampleplot = resample_fare_by_date_df.plot(figsize = (20,6))
# Use the graph style fivethirtyeight.
style.use('fivethirtyeight')
# Add a title 
resampleplot.set_title("Total Fare by City Type")
#  Add a x- and y-axis label.
resampleplot.set_xlabel("")
resampleplot.set_ylabel("Fare ($USD)")
# Plot the resampled DataFrame.
plt.tight_layout()
plt.savefig("analysis/TotalFare_by_CityType.png")
plt.show()